In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
import pandas as pd
import numpy as np
import gc
import matplotlib.ticker as ticker
import matplotlib.pyplot as plt
import plotly.express as px
from pylab import rcParams
import seaborn as sns
sns.set_style('darkgrid')
rcParams['figure.figsize'] = 8,8
import warnings
warnings.filterwarnings("ignore")
%matplotlib inline 
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train_df = pd.read_csv('../input/layeraiairqualitypredictionchallenge/train.csv')
test_df = pd.read_csv("../input/layeraiairqualitypredictionchallenge/test.csv")

In [ ]:
ntrain= train_df.shape[0]
ntest = test_df.shape[0]
df = pd.concat((train_df, test_df)).reset_index(drop=True)

In [ ]:
null_cols = [col for col in df.columns if df[col].isnull().any().sum() > 0]
for col in null_cols:
    df[col] = df.groupby('device')[col].transform(lambda x: x.bfill())
df = df.fillna(0)

In [ ]:
df=df.drop(columns=["ID"])
datetime_series = pd.to_datetime(df['date'])
# create datetime index passing the datetime series
datetime_index = pd.DatetimeIndex(datetime_series.values)
df=df.set_index(datetime_index)
df = df.sort_index()

In [ ]:
df = df.drop(columns="date")
df

In [ ]:

def create_time_features(df):
    df['year'] = df.index.year
    df['month'] =df.index.month
    df['day'] = df.index.day
    df['quarter'] = df.index.quarter
    df["weekday"] = df.index.weekday
    df['day_of_year'] = df.index.dayofyear
    df['week_of_year'] = df.index.weekofyear
    df['season'] = df.month%12 // 3 + 1
    return df
df=create_time_features(df)

In [ ]:
# pm2_5 The target value, well use it to create a dictionary that can aid in mapping the lag features to the dataframe
target_pm = df["pm2_5"].to_dict() 

In [ ]:
def add_lags(df): 
    df["lag1"]= (df.index - pd.Timedelta('364 days')).map(target_pm)
    df["lag2"]= (df.index - pd.Timedelta('728 days')).map(target_pm)
    return df
df = add_lags(df)

In [ ]:
df["lag1"]

In [ ]:
14177/10

# ***Cross_Validation***

In [ ]:
from sklearn.model_selection import TimeSeriesSplit

tss = TimeSeriesSplit(n_splits=5, test_size=90*24)
fold=0
preds=[]

In [ ]:
fig, axs = plt.subplots(5, 1, figsize=(15,15), sharex=True)
for train_idx, test_idx in tss.split(df):
    train = df.iloc[train_idx]
    test = df.iloc[test_idx]
    train["pm2_5"].plot(ax=axs[fold], label="Training set", title=f'Data Train/Test split{fold}')
    test["pm2_5"].plot(ax=axs[fold], label="Test set")
    axs[fold].axvline(test.index.min(), color="black", ls="--")
    fold+=1
plt.show()